# Coupling Polymers to MPCD Solvent

In this session, we will couple a polymer model to MPCD solvent. 

## Overview

### Questions

- How do I couple a simple polymer model to MPCD?
- How do hydrodynamic interactions change the diffusion of polymers?
  
### Objectives

- Understand how to couple MD particles to MPCD solvent.
- Set up a simple polymer simulation that now uses MPCD as solvent.
- Compare simulations with MPCD (hydrodynamics) to Langevin (no hydrodynamics). 

## Boilerplate code

In [8]:
import freud
import gsd.hoomd
import hoomd
import hoomd.azplugins
import hoomd.mpcd
import matplotlib
import numpy 
import fresnel 

%matplotlib inline
matplotlib.style.use("ggplot")
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

def render(frame):
    scene = fresnel.Scene()
    geometry = fresnel.geometry.Sphere(scene, N=frame.particles.N)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]), roughness=0.5)
    geometry.position[:] = frame.particles.position
    geometry.outline_width = 0.01
    box = fresnel.geometry.Box(scene, frame.configuration.box,box_radius=0.01)
    L = frame.configuration.box[0]
    scene.camera = fresnel.camera.Perspective(position=(L*1.8, L*1.8, L * 2.2), look_at=(0, 0, 0), up=(0, 1, 0), height=0.28)

    if frame.bonds.N>0:
        geometry.radius[:] = [0.2]*frame.particles.N 

        all_bonds = numpy.stack(
        [
            frame.particles.position[frame.bonds.group[:, 0]],
            frame.particles.position[frame.bonds.group[:, 1]],
        ],
        axis=1,
        )
        # Use a distance cutoff (L/2) to filter bonds that span the periodic boundary
        bond_distances = numpy.linalg.norm(all_bonds[:,0,:]-all_bonds[:,1,:], axis=1)
        L = frame.configuration.box[0]
        bond_indices = numpy.where(bond_distances < L/2)[0]
        filtered_bonds = all_bonds[bond_indices, :, :]
        
        bonds = fresnel.geometry.Cylinder(scene, N=len(filtered_bonds))
        bonds.material = fresnel.material.Material(roughness=0.5)
        bonds.outline_width = 0.05

        bonds.points[:] = filtered_bonds
        bonds.radius[:] = [0.1]*len(filtered_bonds)
        bonds.material.primitive_color_mix = 1.0
        bonds.color[:] = fresnel.color.linear([0.8, 0.8, 0.8])
        
    return fresnel.preview(scene)


## Background 

We will largely follow [Mussawisade, K., M. Ripoll, R. G. Winkler, and G. Gompper. "Dynamics of polymers in a particle-based mesoscopic solvent." The Journal of chemical physics 123, no. 14 (2005)](http://doi.org/10.1063/1.2041527) for our model/parameter choices. 

## Initialization

While we could simply set up the polymers as rods and make our box very large in the case of a simple non-interacting Gaussian chain with no solvent, this becomes more expensive when using MPCD. The box has to be filled with MPCD particles at the choosen density. For this reason, we have to be a bit more clever about how we initialize the polymer configuration. Often, it might also be desirable to kepp the box size (and/or concentration) fixed, while changing the polymer length. 

For this purpose, we will use a random walk to intialize the polymer positions instead of a straight rod: 

In [9]:
def random_walk(N,origin):
    vec = numpy.random.randn(3, N-1)
    vec /= numpy.linalg.norm(vec, axis=0)
    # concatenate the random numbers to make a polymer configuration
    path = numpy.concatenate([origin.reshape(1,3), vec.T]).cumsum(0)
    return path

In [ ]:
num_pol = 10
num_mon = 80
n = 0 
L = 25

pos_pol = []
while n<num_pol:
    # draw a random walk starting from a random origin
    origin = ...
    d = random_walk(num_mon,....)
    # check if monomers are inside of the box 
    if not(numpy.any(d > L/2) or numpy.any(d <-L/2)):
        # if yes, add to position array, if no, keep going in while loop
        pos_pol.append(...)
        n+=1

positions = ....
bonds = ...

frame = gsd.hoomd.Frame()
frame.particles.types = ['monomers']
frame.particles.N = ...
frame.particles.position = ...
frame.bonds.N = ..
frame.bonds.group = ...
frame.bonds.types = ['b']
frame.configuration.box = [L, L, L, 0, 0, 0]

We want to make sure all the MD particles (monomers) have the right mass and no drift in
their velocities. We set the mass of the MD particles to be the same as the mass
of MPCD particles in a collision cell. The collisions cells have unit volume by
default, so this mass is same as the number density, $\rho = 5/\ell^3$, times
the MPCD particle mass *m*. We zero all the velocities of the MD particles, because we will draw the correct velocities for the MPCD particles later. The MPCD particles will then re-thermalize the MD particles very quickly.

In [ ]:
mass = 1
density = 5
kT = 1

frame.particles.mass = [mass * density]*len(positions)
frame.particles.velocity = [0,0,0]*len(positions)

render(frame)

Now we can add the MPCD particles. The `gsd.hoomd.Frame` does not have a field for the MPCD particles. Usually, you do **not** want to write all their positions or properties into gsd files anyway, because that tends to be a lot of data. To circumvent this issue, we will simply read in the MD (monomer) configuration into hoomd-blue, then immediately take a snapshot via `simulation.state.get_snapshot()`. To this `snapshot` object, we can then add the MPCD solvent information. 

Draw random uniform positions and velocities from the correct Maxwell Boltzman distribution for the MPCD particles: 

In [ ]:
# set up simulation
# load frame 
# take snapshot 
... 

# set mpcd properties

snapshot.mpcd.types = ["A"]
snapshot.mpcd.N = ...
snapshot.mpcd.mass = ...
snapshot.mpcd.position[:] = ...
snapshot.mpcd.velocity[:] =...

# load modified snapshot containing MPCD particles 
simulation.state.set_snapshot(snapshot)


## Configuring the MPCD integrator

Since we have regular HOOMD particles that are embedded in our MPCD particles,
we will need to configure both MD and MPCD options.

### MD configuration

We need to use a timestep that is suitable for MD. Here, we are going use a timestep of $0.01\,\tau$, following the original paper. If a simulation uses stiffer bonds or pair interactions, the timestep needs to be adjusted accordingly!

In [ ]:
integrator = hoomd.mpcd.Integrator(dt=0.01)
simulation.operations.integrator = integrator

Since we want to simulate the same polymer model as before, we only need to define harmonic springs as we did before. We rename the the root-mean-square bond length variable to $b$ (from $l$ used in the original paper), so that it doesn't conflict with the MPCD collison cell size $l$, which is the unit of length. 

In [ ]:
# add harmonic potential to simulation (same model as for session 8)

We will use a `ConstantVolume` (NVE) integration method
for the MD particles (monomers). They will be thermostatted through the MPCD particles, so **no** additional/seperate thermostat is needed for the MD particles. 

In [ ]:
nve = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
integrator.methods.append(nve)

### MPCD configuration

Now that the base MD simulation is configured, we can setup the corresponding
MPCD simulation. The collision method will be our standard [SRD fluid](https://hoomd-blue.readthedocs.io/en/v5.3.1/hoomd/mpcd/collide/stochasticrotationdynamics.html)
(collision time $\Delta t = 0.1\tau$, collision angle $\alpha = 130^\circ$, and
a thermostat to maintain constant temperature), but **importantly, we now couple
the MD particles (monomers) to the solvent via `embedded_particles`!** 

In [ ]:
# copy the collision method from previous session, add "embedded_particles" 
...

The streaming method will be `Bulk`:

In [ ]:
# copy streaming method 
...

Last, we make sure we don't forget our sorter for performance.

In [ ]:
# copy particle sorter 
....

Now we can run the simulation. Like before, we equilibrate for 100,000 steps and then run for 5 million. We write the results into a gsd file named `'run_len_%s_pol_%s.gsd'%(num_mon,num_pol)`. Don't forget to record the images!

In [ ]:
# Equilibrate 

simulation.run(100_000)

# create and add GSD writer
gsd_out = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(1_000), 
    mode='wb',
    dynamic=['property','momentum'],
    filename='run_len_%s_pol_%s.gsd'%(num_mon,num_pol))

simulation.operations.writers.append(gsd_out)

# run
simulation.run(5_000_000)

gsd_out.flush()
render(simulation.state.get_snapshot())

## Diffusion Coefficient 

Let's calculate the diffusion coefficient from the simulation. The code is the same as for "Session 8: PBC -- dynamics". Write a function that takes a `filename` and a `timestep` and returns arrays containing the lagtimes, all three MSDs (`msd1, msd2, msd3`), diffusion coefficient over time `D` and an average diffusion coefficient `av_D`. 

In [37]:
def calculate_D(filename,timestep):
    # add code 
    ...
    return lagtimes, D, av_D, msd3, msd2, msd1

For the one simulation you have done above, plot the results. Plot all three MSDs in one panel, and the diffusion coefficient over time. Do you need to ajust your `fit_range` choice in the above function? 

In [ ]:
# add code (same as Session 8)
....

Repeat your simulation for a few chain lengths between 20 and 80. Plot all $MSD(t)$ and $D(t)$ results:

In [ ]:
# add code (either re-execute simulation cell from above (don't forget to re-initialize the frame!) or copy paste the simulation code here and change the chain length. 
.... 

For convinience, there is a file called "mpcd-gaussian-D-Rg.txt" that contains chain lengths `N`, `av_D`, and `Rg` values for this model. Compute average diffusion constants for all simulations you have done and compare them to the results in this file. What **scaling** do they follow? What is the theoretical expectation? *Hint:* Look up Zimm model. **Compare the MPCD simulation results to the Langevin simulation results**.  

In [ ]:
# add code (Same as in Session 8, just the file name is different) 
# also re-plot your results from Session 8 to compare
....


## Additions 

1. Compare the radius of gyration of the Gaussian model with and without hydrodynamics. Do you expect a difference?
2. The diffusion coefficients not only show different scaling, but also have different absolute values. What parameters could we change to influence the diffusion coefficient in each case?
3. Implement a model that uses MPCD and has excluded-volume interactions. What do we need to change? How would we set up the polymer chains? How do we tune the solvent quality in such a model? 